<a href="https://colab.research.google.com/github/18521039/covid19_fakenew-dectected/blob/main/DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd drive/My Drive/ĐỒ ÁN ML

In [ ]:
import pandas as pd

dataset = pd.read_excel('Constraint_English_Train.xlsx')
dataset.loc[:,'label'] = dataset.label.map({'real':0, 'fake':1})
Y_train = dataset.iloc[:,-1]

datatest = pd.read_excel('Constraint_English_Val.xlsx')
datatest.loc[:,'label'] = datatest.label.map({'real':0, 'fake':1})
Y_Val = datatest.iloc[:,-1]

In [ ]:
dataset

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import re
from gensim.parsing.preprocessing import strip_non_alphanum
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus_train = []
for tweet in dataset.values[:,1]:
  tweet = re.sub('http\S+', '', tweet) 
  tweet = re.sub('[^a-zA-Z0-9]', ' ', tweet)
  tweet = strip_non_alphanum(tweet).lower().strip() 
  tweet = tweet.split() 
  ps = PorterStemmer()
  tweet = [ps.stem(word) for word in tweet if word not in stopwords.words("english")]
  tweet = " ".join(tweet)
  corpus_train.append(tweet)

corpus_test = []
for tweet in datatest.values[:,1]:
  tweet = re.sub('http\S+', '', tweet) 
  tweet = re.sub('[^a-zA-Z0-9]', ' ', tweet)
  tweet = strip_non_alphanum(tweet).lower().strip() 
  tweet = tweet.split() 
  ps = PorterStemmer()
  tweet = [ps.stem(word) for word in tweet if word not in stopwords.words("english")]
  tweet = " ".join(tweet)
  corpus_test.append(tweet)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(corpus_train))

x_tr_seq  = tokenizer.texts_to_sequences(corpus_train) 
x_val_seq = tokenizer.texts_to_sequences(corpus_test)

x_tr_seq  = pad_sequences(x_tr_seq, maxlen=20)
x_val_seq = pad_sequences(x_val_seq, maxlen=20)
print(x_tr_seq)
size_of_vocabulary=len(tokenizer.word_index) + 1
print(size_of_vocabulary)

In [ ]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
import keras
from keras.models import *
from keras.layers import *
from keras.callbacks import *
import numpy as np

model=Sequential()

model.add(Embedding(size_of_vocabulary,300,input_length=20,trainable=True)) 

model.add(LSTM(128,return_sequences=True,dropout=0.2))

model.add(GlobalMaxPooling1D())

model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='sigmoid')) 

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['acc',f1_m, keras.metrics.Recall(), keras.metrics.Precision()]) 

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

print(model.summary())

history = model.fit(np.array(x_tr_seq),np.array(Y_train),batch_size=128,epochs=10,validation_data=(np.array(x_val_seq),np.array(Y_Val)),verbose=1,callbacks=[mc,es])